<a href="https://colab.research.google.com/github/IyadKhuder/TextClassification_TensorFlowHub/blob/main/TextClassification_TensorFlowHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Hub - Text classification

- Based on: https://www.tensorflow.org/hub/tutorials/tf2_text_classification

# Importing the libraries

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print('TensorFlow version: ', tf.__version__)
print('TensorFlow Hub version: ', hub.__version__)

TensorFlow version:  2.9.2
TensorFlow Hub version:  0.12.0


# Dataset

- Extracted from imdb: 0 - negative, 1 - positive

In [3]:
train_data, test_data = tfds.load(name = 'imdb_reviews', split =['train', 'test'], batch_size=-1, as_supervised=True)
X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete03BX97/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete03BX97/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete03BX97/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
X_train.shape, y_train.shape

((25000,), (25000,))

In [5]:
X_test.shape, y_test.shape

((25000,), (25000,))

In [7]:
X_train[0:2]

array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [8]:
y_train[0:2]

array([0, 0])

In [9]:
X_test[0:2]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [10]:
y_test[0:2]

array([1, 1])

In [11]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([12500, 12500]))

In [12]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([12500, 12500]))

# Building and training the neural network

In [29]:
model_path = 'https://tfhub.dev/google/nnlm-en-dim50/2'

In [14]:
embedding_layer = hub.KerasLayer(model_path, input_shape = [], dtype = tf.string, trainable = True)

In [30]:
embedding_layer(X_train[0:2])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.43250495, -0.10018105,  0.23472811,  0.11847009, -0.15165758,
        -0.3773563 ,  0.2216346 ,  0.27437198, -0.23451936, -0.19802941,
         0.3555607 ,  0.2192843 ,  0.07330744,  0.48308334, -0.23220272,
        -0.19698939,  0.05512122, -0.02179846,  0.41175762, -1.4207407 ,
         0.12905295, -0.19712113,  0.673627  , -0.13756259, -0.46192056,
        -0.01752323, -1.1702346 , -0.09275448,  0.533089  , -0.8542099 ,
        -0.7178316 ,  0.15252452,  0.6596266 , -0.2713876 , -0.29616588,
         0.04112098,  0.27608776, -0.17804871,  0.33124286, -0.6282892 ,
        -0.2778648 ,  0.06024722, -0.34294453,  0.19739507,  0.15456067,
        -0.5426914 , -0.05445153, -0.48710442,  0.09970611, -0.61058414],
       [ 0.5150139 , -0.13745084,  0.02562648,  0.75458986, -0.176223  ,
        -0.34253323, -0.02810713,  0.18751758, -0.3789552 ,  0.08213472,
         0.02151299,  0.18998414, -0.07851301,  0.33501697, -0.19357339,
 

In [18]:
model = tf.keras.Sequential()
model.add(embedding_layer)
# 50 -> 16 -> 1
model.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer = 'adam', loss = tf.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs = 10, batch_size = 512, verbose = 1)

Epoch 1/10
49/49 [==============================] - 4s 37ms/step - loss: 0.6258 - accuracy: 0.5785
Epoch 2/10
49/49 [==============================] - 2s 37ms/step - loss: 0.4342 - accuracy: 0.7944
Epoch 3/10
49/49 [==============================] - 2s 37ms/step - loss: 0.2820 - accuracy: 0.8839
Epoch 4/10
49/49 [==============================] - 2s 36ms/step - loss: 0.1967 - accuracy: 0.9254
Epoch 5/10
49/49 [==============================] - 2s 36ms/step - loss: 0.1406 - accuracy: 0.9529
Epoch 6/10
49/49 [==============================] - 2s 37ms/step - loss: 0.1003 - accuracy: 0.9694
Epoch 7/10
49/49 [==============================] - 2s 37ms/step - loss: 0.0707 - accuracy: 0.9823
Epoch 8/10
49/49 [==============================] - 2s 37ms/step - loss: 0.0498 - accuracy: 0.9892
Epoch 9/10
49/49 [==============================] - 2s 37ms/step - loss: 0.0349 - accuracy: 0.9945
Epoch 10/10
49/49 [==============================] - 2s 37ms/step - loss: 0.0248 - accuracy: 0.9974


In [21]:
results = model.evaluate(X_test, y_test)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.4084 - accuracy: 0.8533
[0.40844082832336426, 0.8533200025558472]


# Predictions

In [22]:
X_test[0:5]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [23]:
y_test[0:5]

array([1, 1, 0, 0, 1])

In [24]:
predictions = model.predict(X_test[0:5])

1/1 [==============================] - 0s 81ms/step


In [25]:
predictions

array([[ 0.57704127],
       [-1.6190056 ],
       [-4.024676  ],
       [-6.290837  ],
       [ 6.916109  ]], dtype=float32)

In [26]:
predictions = tf.nn.sigmoid(predictions).numpy()
predictions

array([[0.64038634],
       [0.16534205],
       [0.01755551],
       [0.00184978],
       [0.99900925]], dtype=float32)

In [27]:
y_test[0:5]

array([1, 1, 0, 0, 1])

In [28]:
predictions = (predictions >= 0.5)
predictions

array([[ True],
       [False],
       [False],
       [False],
       [ True]])